In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import json
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from preprocessing import *

In [3]:
'''전처리 결과'''
seed = 99
tf.random.set_seed(seed)

# 인코더의 입력값
index_inputs = np.load(open('data_in/train_inputs.npy','rb'), allow_pickle=True)
# 디코더의 입력값
index_outputs = np.load(open('data_in/train_outputs.npy','rb'), allow_pickle=True)
# 디코더의 타깃값
index_targets = np.load(open('data_in/train_targets.npy','rb'), allow_pickle=True)
# dictonary
prepro_configs = json.load(open('data_in/data_configs.json'))

'''
인코더 Input : 최대 길이만큼 <PAD>
디코더 Input : 시작을 알리는 <SOS>
디코더 타겟 : 끝을 알리는 <END>
'''



'\n인코더 Input : 최대 길이만큼 <PAD>\n디코더 Input : 시작을 알리는 <SOS>\n디코더 타겟 : 끝을 알리는 <END>\n'

In [4]:
BATCH_SIZE = 2  # set을 키워보자 -> NoneType 에러가 발생한다. - 메모리이슈
MAX_SEQUENCE =25
EPOCH =30
UNITS =1024
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.1

char2idx = prepro_configs['char2idx']
idx2char = prepro_configs['idx2char']
std_index = prepro_configs['std_symbol']
end_index = prepro_configs['end_symbol']
vocab_size = prepro_configs['vocab_size']



In [22]:
'''
스케일 내적 어텐션
softmax(Q,k/크기)*value
Scaling을 해주는 이유는 query, value를 이용해 내적한 값이 벡터 차원이 커지면
학습이 잘 안될 수도 있기 때문에 벡터 크기에 따라 값이 반비례하도록 크기를 조정함


'''

def create_look_ahead_mask(size):
    mask= 1 - tf.linalg.band_part(tf.ones((size,size)),-1,0)
    return mask

def scaled_dot_product_attention(q,k,v,mask=None):
    matmul_qk = tf.matmul(q,k,transpose_b = True)
    dk = tf.cast(tf.shape(k)[-1],tf.float32) # Type을 변환함
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
    if mask is not None:
#         print(scaled_attention_logits)
#         print(mask)
        # scaled_attention_logits 이 상삼각행렬이 0인가?
        scaled_attention_logits += (mask * -1e9) # 마스킹 대상에  모두 작은 음수값을 넣는 것  매우 작아지는구나
#         print(scaled_attention_logits)
    
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)
    output = tf.matmul(attention_weights,v)
    
    return output, attention_weights

    

In [23]:
# test
x=[[1.,2.,3.],[3.,4.,5],[5.,6.,7.]]
mask = create_look_ahead_mask(3)
scaled_dot_product_attention(x,x,x,mask)

(<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[1.      , 2.      , 3.      ],
        [2.999998, 3.999998, 4.999998],
        [5.      , 6.      , 7.      ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [9.5992834e-07, 9.9999905e-01, 0.0000000e+00],
        [8.8453794e-19, 9.4049879e-10, 1.0000000e+00]], dtype=float32)>)

In [24]:
'''
멀티 헤드 어텐션 
'''
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,**kargs):
        super(MultiHeadAttention,sefl).__init__()
        self.num_heads = kargs['num_heads']
        self.d_model = kargs['d_model']
        
        # d_model : Q,K,V 차원을 결정하는 Parameter
        # num_heads : 어텐션 head 수를 결정하는 parameter
        # assert는 아래 조건에 해당하지 않으면 에러를 발생시킴
        assert self.d_model % self.num_heads ==0 # 나머지가 없어야한다.
        
        self.depth = self.d_model // self.num_heads #각 Head에 입력될 벡터 차원 수
        
        self.wq = tf.keras.layers.Dense(kargs['d_model'])
        self.wk = tf.keras.layers.Dense(kargs['d_model'])
        self.wv = tf.keras.layers.Dense(kargs['d_model'])
        
        self.dense = tf.keras.layers.Dense(kargs['d_model'])
        
    def split_heads(self, x, batch_size): # 학습 중에 배치 크기가 바뀔 수 있음.
        # (batch, sequence, feature) → (batch, head, sequence, feature)
        x = tf.reshape(x, (batch_size,-1,self.num_heads,self.depth))
        return tf.transpose(x, perm = [0,2,1,3]) # Sequence, head 차원을 바꿈
    
    def call(self, v, k, q, mask=None):
        batch_size = tf.shape(q)[0]
        
        q= self.wq(q)
        k= self.wk(k)
        v = self.wv(v)
        
        q = self.split_heads(q,batch_size)
        k = self.split_heads(k,batch_size)
        v = self.split_heads(v,batch_size)
        
        scaled_attention, attention_weights = scaled_dot_product_attention(q,k,v,mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm = [0,2,1,3]) # (batch, seq, feature) 차원
        # feature 
        concat_attention = tf.reshape(scaled_attention,(batch_size,-1, self.d_model))
        
        output = self.dense(concat_attention) # 멀티헤드 어텐션 벡터 
        
        return output, attention_weights
        



In [25]:
'''
Position-wise FFNN
'''
def point_wise_feed_forward_network(d_model,dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff,activation = 'relu'),
        tf.keras.layers.Dense(d_model)        
    ])



In [27]:
'''
Positional Encoding
'''

def get_angles(pos,i,d_model):
    angle_rates = 1 / np.power(10000, (2*i) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    # 포지션과 차원별로 각기 다른 값을 순차적으로 할당
    angle_rads = get_angles(np.arange(position)[:,np.newaxis],
                           np.arange(d_model)[np.newaxis,:],
                            d_model
                           )
    
    # 짝수차원에는 사인 함수, 홀수 차원에는 코사인 함수를 적용 
    angle_rads[:,0::2] = np.sin(angle_rads[:,0::2])
    angle_rads[:,1::2] = np.cos(angle_rads[:,1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype= tf.float32)

In [28]:
'''
인코더
'''
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(**kargs)
        
        self.ffn = point_wise_feed_forward_network(**kargs)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])
        
    def call(self, x, mask=None):
        attn_output, _ = self.mha(x,x,x,mask)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)  # x를 그대로 더해줌 Residual Connection

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output) 
        
        return out2
    
        



In [30]:

class Encoder(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(Encoder, self).__init__()
        
        self.d_model = kargs['d_model']
        self.num_layers = kargs['num_layers']
        
        self.embedding = tf.keras.layers.Embedding(kargs['input_vocab_size'], self.d_model)
        self.pos_encoding = positional_encoding(kargs['maximum_position_encoding'], self.d_model)
        
        self.enc_layers = [EncoderLayer(**kargs) for _ in range(self.num_layers)]
        self.dropout = tf.keras.layers.Dropout(kargs['rate'])
        
    def call(self,x,mask=None):
        seq_len = tf.shape(x)[1] # 포지션 임베딩을 위함
        # word embedding은  입력 길이가 가변적이고 포지션 임베딩인 고정이기 때문
        
        x= self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model,tf.float32)) # 임베딩에 대한 스케일을 맞추는 것
        # 임베딩 차원의 제곱근 만큼 가중치 곱함
        x += self.pos_encoding[:,:seq_len,:]
        
        x = self.dropout(x)
        
        for i in range(self.num_layers):
            x = self.enc_layers[i](x,mask)
            
        return x
    
        

In [ ]:
'''
Decoder

'''